# Dynamic Customer **Support RAG chatbot** 

This notebook demonstrates:

- Document ingestion
- Embedding generation
- FAISS vector indexing 
- Content retrieval 
- LLM-Based answer generation
- Incremental knowledge update

## ***Imports***

In [1]:
import os 
from pathlib import Path
from dotenv import load_dotenv

from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq

# Loading Environment **Variables**

In [2]:
load_dotenv()

llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.1-8b-instant",
    temperature=0.1
)

## Load **Dataset**

In [3]:
data_path = Path("dataset/et.csv")

loader = CSVLoader(file_path=str(data_path))
documents = loader.load()

print(f"Loaded {len(documents)} Documents.")

Loaded 78 Documents.


## **Embeddings**

In [5]:
embeddings = HuggingFaceBgeEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)

print(" embeddings Model Loaded Sucessfully ")

c:\Users\shriyansh\OneDrive\Desktop\GenAI-main\Extended-Customer-Service-Chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 757.09it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


 embeddings Model Loaded Sucessfully 


## Created **FAISS** Index

In [6]:
vectordb = FAISS.from_documents(documents,embeddings)

print("FAISS Index Created Sucessfully")

FAISS Index Created Sucessfully


## Created **Retriever** 

In [7]:
retriever = vectordb.as_retriever(search_kwargs = {"k": 3})

## Created **Prompt** Template

In [8]:
prompt = PromptTemplate(
    template="""
    Answer the question ONLY using the context below.
    If the answer is not found, say "I don't know."
    
    CONTEXT
    {context}

    QUESTION
    {question}
    """,

    input_variables=["context","question"]
)

## Build **RAG** Chain

In [9]:
chain = (
    {
        "context" : retriever,
        "question" : RunnablePassthrough()
    }

    | prompt
    | llm

)

## Explanation Of The Above Strcture

- "context" : retriever :- It Retrieves Relevant Documents based on User's Question From Vector Database.
- RunnablePassthrough() :- Forwards the original question unchanged so it can be used later in the prompt.
- prompt :-  combines both the retrieved context and original question into structured Instuction fot the model . 
- LLM :- Generates the final response using both the contextual information and the user’s query. 

## Demo Of The **chatbot**

- One Question

In [14]:
question = "WHAT IS  NULL CLASS"

response = chain.invoke(question)

print(response.content)

This Is a Trusted Platform For Anyone Who wants To Learn about Coding And Tech.


## Asking another **question** 

In [19]:
question_2 = "Is there any prerequisite for taking this bootcamp ?"

response2 = chain.invoke(question_2)

print(response2.content)

Our bootcamp is specifically designed for beginners with no prior experience in this field. The only prerequisite is that you need to have a functional laptop with at least 4GB ram, an internet connection, and a thrill to learn data science.


# Showing Retrieved **Content**

In [26]:
docs = retriever.get_relevant_documents("What services does NullClass provide?")
                                        
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:300])


Document 1:
prompt: What is Nullclass
response: This Is a Trusted Platform For Anyone Who wants To Learn about Coding And Tech

Document 2:
prompt: Why should I trust Nullclass?
response: Till now 9000 + learners have benefitted from the quality of our courses. You can check the review section and also we have attached their LinkedIn profiles so that you can connect with them and ask directly.

Document 3:
prompt: who are you
response: Hi there, I am your virtual assistant developed by Nullclass, I'm here to provide response for your queries related to Nullclass.
